# Generating nodes for GML

In [1]:
import pandas as pd

In [2]:
def get_state_with_connections_from_csv(file_path):
    state_with_connections_df = pd.read_csv(file_path, header=None)
    output_list = state_with_connections_df.values.tolist()
    return output_list

csv_path = "C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/hugo_test/state_with_connections.csv"
state_with_connections_output = get_state_with_connections_from_csv(csv_path)
print(state_with_connections_output)

[['C1', 'N1', '51.60,50.15,50.40', 'T1'], ['C2', 'N2', '151.60,50.15,50.40', 'T2'], ['B1', 'SD1', '100.00,50.00,20.00', 'T1,T2']]


In [3]:
#new code for generating nodes GML with connected edges 
def generate_state_gml(data):
    xml_output = """  <multiLayeredGraph>
    <MultiLayeredGraph gml:id="MLG_s76">
      <gml:boundedBy xsi:nil="true"/>
      <spaceLayers gml:id="NRG0">
        <gml:boundedBy xsi:nil="true"/>
        <spaceLayerMember>
          <SpaceLayer gml:id="SLs_40">
            <gml:boundedBy xsi:nil="true"/>
            <nodes gml:id='SLs_40-nodes'>
              <gml:boundedBy xsi:nil="true"/>"""
    for item in data:
        cell_id, state_id, coordinates, transitions = item
        transitions = transitions.split(',')
        formatted_coordinates = coordinates.replace(',', ' ')
        xml_output += f"""
              <stateMember>
                <State gml:id="{state_id}">
                  <gml:boundedBy xsi:nil="true"/>
                  <duality xlink:href="#{cell_id}"/>"""
        for transition in transitions:
            xml_output += f"""
                  <connects xlink:href="#{transition}"/>"""
        xml_output += f"""
                  <geometry>
                    <gml:Point gml:id="SG-{state_id}">
                      <gml:pos srsDimension="3">{formatted_coordinates}</gml:pos>
                    </gml:Point>
                  </geometry>
                </State>
              </stateMember>"""
    xml_output += """
            </nodes>"""

    return xml_output

state_gml = generate_state_gml(state_with_connections_output)
print(state_gml)

  <multiLayeredGraph>
    <MultiLayeredGraph gml:id="MLG_s76">
      <gml:boundedBy xsi:nil="true"/>
      <spaceLayers gml:id="NRG0">
        <gml:boundedBy xsi:nil="true"/>
        <spaceLayerMember>
          <SpaceLayer gml:id="SLs_40">
            <gml:boundedBy xsi:nil="true"/>
            <nodes gml:id='SLs_40-nodes'>
              <gml:boundedBy xsi:nil="true"/>
              <stateMember>
                <State gml:id="N1">
                  <gml:boundedBy xsi:nil="true"/>
                  <duality xlink:href="#C1"/>
                  <connects xlink:href="#T1"/>
                  <geometry>
                    <gml:Point gml:id="SG-N1">
                      <gml:pos srsDimension="3">51.60 50.15 50.40</gml:pos>
                    </gml:Point>
                  </geometry>
                </State>
              </stateMember>
              <stateMember>
                <State gml:id="N2">
                  <gml:boundedBy xsi:nil="true"/>
                  <duality xlink:href

# Generating edges for GML

In [4]:
import csv

def create_edges_list_from_csv(file_path):
    edges_list = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader, None)  # Skip the header row if present
        for row in csv_reader:
            transition_id = row[0]
            weight = int(row[1])
            start_node = row[2]
            end_node = row[3]
            coordinates = ','.join(row[4:])
            edges_list.append([transition_id, weight, start_node, end_node, coordinates])
    return edges_list

file_path = "C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/hugo_test/edges.csv"  # Update this path to your actual file path
edges_list = create_edges_list_from_csv(file_path)
print(edges_list)


[['T1', 1, 'N1', 'SD1', '51.60,50.15,50.40,100.00,50.00,20.00'], ['T2', 1, 'SD1', 'N2', '100.00,50.00,20.00,151.60,50.15,50.40']]


In [5]:
list_of_lists = [
    ['T1', 1, 'N1', 'SD1', '51.60,50.15,50.40,100.00,50.00,20.00'],
    ['T2', 1, 'SD1', 'N2', '100.00,50.00,20.00,151.60,50.15,50.40']
]

def generate_xml_from_list(data):
    xml_output = """            <edges gml:id="SLs_40-edges">
              <gml:boundedBy xsi:nil="true"/>\n"""
    
    for item in data:
        transition_id, weight, start_node, end_node, coordinates = item
        coords = coordinates.split(',')
        xml_output += f"""              <transitionMember>
                <Transition gml:id="{transition_id}">
                  <gml:boundedBy xsi:nil="true"/>
                  <weight>{float(weight):.1f}</weight>
                  <connects xlink:href="#{start_node}"/>
                  <connects xlink:href="#{end_node}"/>
                  <geometry>
                    <gml:LineString gml:id="TG-{transition_id}">\n"""
        for i in range(0, len(coords), 3):
            xml_output += f"""                      <gml:pos srsDimension="3">{coords[i]} {coords[i+1]} {coords[i+2]}</gml:pos>\n"""
        xml_output += """                    </gml:LineString>
                  </geometry> 
                </Transition>
              </transitionMember>\n"""
    
    xml_output += """            </edges>
          </SpaceLayer>
        </spaceLayerMember>
      </spaceLayers>
    </MultiLayeredGraph>
  </multiLayeredGraph>
</IndoorFeatures>"""
    
    return xml_output

# Generate and print the XML text
xml_text = generate_xml_from_list(edges_list)
print(xml_text)


            <edges gml:id="SLs_40-edges">
              <gml:boundedBy xsi:nil="true"/>
              <transitionMember>
                <Transition gml:id="T1">
                  <gml:boundedBy xsi:nil="true"/>
                  <weight>1.0</weight>
                  <connects xlink:href="#N1"/>
                  <connects xlink:href="#SD1"/>
                  <geometry>
                    <gml:LineString gml:id="TG-T1">
                      <gml:pos srsDimension="3">51.60 50.15 50.40</gml:pos>
                      <gml:pos srsDimension="3">100.00 50.00 20.00</gml:pos>
                    </gml:LineString>
                  </geometry> 
                </Transition>
              </transitionMember>
              <transitionMember>
                <Transition gml:id="T2">
                  <gml:boundedBy xsi:nil="true"/>
                  <weight>1.0</weight>
                  <connects xlink:href="#SD1"/>
                  <connects xlink:href="#N2"/>
                  <geometry>


In [6]:
import import_ipynb
from processing_cs_and_csb import final_text

importing Jupyter notebook from processing_cs_and_csb.ipynb
C1,N1,1,Room 1
<class 'str'>
0,0,0,100,0,0,100,100,0,0,100,0,0,0,0
<class 'str'>
0,0,100,100,0,100,100,100,100,0,100,100,0,0,100
<class 'str'>
0,0,0,100,0,0,100,0,100,0,0,100,0,0,0,100,0,0,100,100,0,100,100,100,100,0,100,100,0,0,100,100,0,0,100,0,0,100,100,100,100,100,100,100,0,0,100,0,0,0,0,0,0,100,0,100,100,0,100,0
<class 'str'>
[['C1,N1,100,Room1', '0,0,0,100,0,0,100,100,0,0,100,0,0,0,0', '0,0,100,100,0,100,100,100,100,0,100,100,0,0,100', '0,0,0,100,0,0,100,0,100,0,0,100,0,0,0,100,0,0,100,100,0,100,100,100,100,0,100,100,0,0,100,100,0,0,100,0,0,100,100,100,100,100,100,100,0,0,100,0,0,0,0,0,0,100,0,100,100,0,100,0'], ['C2,N2,100,Room2', '100,0,0,200,0,0,200,100,0,100,100,0,100,0,0', '100,0,100,200,0,100,200,100,100,100,100,100,100,0,100', '100,0,0,200,0,0,200,0,100,100,0,100,100,0,0,200,0,0,200,100,0,200,100,100,200,0,100,200,0,0,200,100,0,100,100,0,100,100,100,200,100,100,200,100,0,100,100,0,100,0,0,100,0,100,100,100,100,100

In [7]:
final_gml_output = final_text+'\n' +state_gml+'\n'+xml_text
print(final_gml_output)

<?xml version="1.0" encoding="UTF-8"?>
<IndoorFeatures xmlns:gml="http://www.opengis.net/gml/3.2" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
xmlns="http://www.opengis.net/indoorgml/1.0/core" xsi:schemaLocation="http://www.opengis.net/indoorgml/1.0/core indoorgmlcore.xsd" gml:id="IFs">
  <gml:boundedBy xsi:nil="true"/>
  <primalSpaceFeatures>
    <PrimalSpaceFeatures gml:id="PS1">
      <gml:boundedBy xsi:nil="true"/>
      <cellSpaceMember>
        <CellSpace gml:id="C1">
          <gml:name>Room1</gml:name>
          <gml:area>100</gml:area>
          <gml:boundedBy xsi:nil="true"/>
          <cellSpaceGeometry>
            <Geometry3D>
              <gml:Solid gml:id="CG-C1">  
                <gml:exterior>
                  <gml:Shell>
                    <gml:surfaceMember>
                      <gml:Polygon>
                        <gml:exterior>
                          <gml:LinearRing>
                            <gml:pos 

In [9]:
output_file_path = "C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/hugo_test/test_output.xml"

# Write the XML content to the file
with open(output_file_path, 'w') as file:
    file.write(final_gml_output)